In [109]:
# 불필요한 Warning 제거
import warnings
warnings.filterwarnings('ignore')

In [110]:
import json

with open("hf_key.json") as f:
    hf_key_ = (json.load(f)["hf_key"])

In [111]:
# !pip install blobfile

In [112]:
!wget https://raw.githubusercontent.com/meta-llama/llama-models/main/models/llama3/api/tokenizer.model

--2024-12-07 00:04:59--  https://raw.githubusercontent.com/meta-llama/llama-models/main/models/llama3/api/tokenizer.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2183982 (2.1M) [text/plain]
Saving to: ‘tokenizer.model.1’

tokenizer.model.1   100%[===================>]   2.08M  2.72MB/s    in 0.8s    

2024-12-07 00:04:59 (2.72 MB/s) - ‘tokenizer.model.1’ saved [2183982/2183982]



In [113]:
import json

import os
from huggingface_hub import InferenceClient # 다운로드 안받고 warm을 호출 gpu 가 필요 없다

In [114]:

def llama3(prompt, temperature=0, raw=False, debug=False):

    print("prompt == ", prompt)

    client = InferenceClient(api_key= hf_key_ )
    for message in client.chat_completion(
      # model="meta-llama/Llama-3.2-11B-Vision-Instruct",
      # model="meta-llama/Llama-3.1-8B-Instruct",
      model="meta-llama/Meta-Llama-3-8B-Instruct",
    #	messages=[{"role": "user", "content": "What is the capital of France?"}],
      messages=[{"role" : "user", "content" : prompt}],
      max_tokens=500,
      stream=True,
    ):
        print(message.choices[0].delta.content, end="")



# pretty print JSON with syntax highlighting
import json
from pygments import highlight, lexers, formatters

def cprint(response):
    formatted_json = json.dumps(response, indent=4)
    colorful_json = highlight(formatted_json,
                              lexers.JsonLexer(),
                              formatters.TerminalFormatter())
    print(colorful_json)


def html_tokens(tokens):
  # simulate the color values used in https://tiktokenizer.vercel.app
  on_colors = ["#ADE0FC", "#FCE278", "#B2D1FE", "#AFF7C6", "#FDCE9B", "#97F1FB", "#DEE1E7", "#E3C9FF", "#BBC6FD", "#D1FB8C"]

  # Create an HTML string with colored spans
  html_string = ""
  for i, t in enumerate(tokens):
      if t == "\n":
            t = "\\n"
      elif t == "\n\n":
            t = "\\n\\n"
      on_col = on_colors[i % len(on_colors)]
      html_string += f'<span style="color: black; background-color: {on_col}; padding: 2px;">{t}</span>'

  return html_string


In [115]:
from pathlib import Path
import tiktoken
from tiktoken.load import load_tiktoken_bpe
import torch
import json
import matplotlib.pyplot as plt

tokenizer_path = "./tokenizer.model"
num_reserved_special_tokens = 256

mergeable_ranks = load_tiktoken_bpe(tokenizer_path)

num_base_tokens = len(mergeable_ranks)
special_tokens = [
    "<|begin_of_text|>",
    "<|end_of_text|>",
    "<|reserved_special_token_0|>",
    "<|reserved_special_token_1|>",
    "<|finetune_right_pad_id|>",
    "<|step_id|>",
    "<|start_header_id|>",
    "<|end_header_id|>",
    "<|eom_id|>",
    "<|eot_id|>",
    "<|python_tag|>",
]
reserved_tokens = [
    f"<|reserved_special_token_{2 + i}|>"
    for i in range(num_reserved_special_tokens - len(special_tokens))
]
special_tokens = special_tokens + reserved_tokens

# source: https://github.com/meta-llama/llama-models/blob/main/models/llama3/api/tokenizer.py#L53
tokenizer = tiktoken.Encoding(
    name=Path(tokenizer_path).name,
    pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
)

In [116]:
from IPython.display import display, HTML

# from utils import html_tokens, llama31

In [117]:
html_tokens(encoded_tokens).split('<span')


['',
 ' style="color: black; background-color: #ADE0FC; padding: 2px;">{\'IQ==\': \'0\'}</span>',
 ' style="color: black; background-color: #FCE278; padding: 2px;">{\'Ig==\': \'1\'}</span>',
 ' style="color: black; background-color: #B2D1FE; padding: 2px;">{\'Iw==\': \'2\'}</span>',
 ' style="color: black; background-color: #AFF7C6; padding: 2px;">{\'JA==\': \'3\'}</span>',
 ' style="color: black; background-color: #FDCE9B; padding: 2px;">{\'JQ==\': \'4\'}</span>',
 ' style="color: black; background-color: #97F1FB; padding: 2px;">{\'Jg==\': \'5\'}</span>',
 ' style="color: black; background-color: #DEE1E7; padding: 2px;">{\'Jw==\': \'6\'}</span>',
 ' style="color: black; background-color: #E3C9FF; padding: 2px;">{\'KA==\': \'7\'}</span>',
 ' style="color: black; background-color: #BBC6FD; padding: 2px;">{\'KQ==\': \'8\'}</span>',
 ' style="color: black; background-color: #D1FB8C; padding: 2px;">{\'Kg==\': \'9\'}</span>',
 ' style="color: black; background-color: #ADE0FC; padding: 2px;"

In [118]:
prompt = "1 부터 10까지 더하면 결과는 무엇이죠? 차근 차근 계산을 해보아요"
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
for i,j in zip(encoded_tokens,decoded_tokens):
    print(f'{i:06d} : {j}')

000016 : 1
086503 :  부
034961 : 터
000220 :  
000605 : 10
102704 : 까지
102519 :  더
108302 : 하면
099901 :  결과
016969 : 는
118947 :  무엇
013094 : 이
110254 : 죠
000030 : ?
103213 :  차
104152 : 근
103213 :  차
104152 : 근
095303 :  계
086157 : 산
018359 : 을
061816 :  해
042771 : 보
112737 : 아요


In [119]:
display(HTML(html_tokens(encoded_tokens)))
display(HTML(html_tokens(decoded_tokens)))

In [120]:
response = llama3(prompt)
print(response)

prompt ==  1 부터 10까지 더하면 결과는 무엇이죠? 차근 차근 계산을 해보아요
Let's calculate the sum step by step from 1 to 10.

1. 1 + 0 = 1
2. 1 + 1 = 2
3. 2 + 2 = 4
4. 4 + 3 = 7
5. 7 + 4 = 11
6. 11 + 5 = 16
7. 16 + 6 = 22
8. 22 + 7 = 29
9. 29 + 8 = 37
10. 37 + 9 = 46
11. 46 + 10 = 56

So, the result is... 56! 🎉None


### 토크나이저

In [121]:
!head -5 ./tokenizer.model

IQ== 0
Ig== 1
Iw== 2
JA== 3
JQ== 4


In [122]:
import base64

encoded_tokens = []
decoded_byte_tokens = []
decoded_utf8_tokens = []

with open("./tokenizer.model", 'r') as file:
  for i, line in enumerate(file):
    k, v = line.strip().split(' ')
    encoded_tokens.append({k: v})
    decoded_byte_tokens.append({base64.b64decode(k): v})
    decoded_utf8_tokens.append({base64.b64decode(k).decode('utf-8', errors="replace") : v})

In [123]:
list(decoded_byte_tokens)[:5]

[{b'!': '0'}, {b'"': '1'}, {b'#': '2'}, {b'$': '3'}, {b'%': '4'}]

In [124]:
# hello 토큰 찾기
hello_encode_to_base64 = base64.b64encode("hello".encode('utf-8'))
hello_encode_to_base64

b'aGVsbG8='

In [125]:
!grep 'aGVsbG8=' ./tokenizer.model

aGVsbG8= 15339
